In [1]:
import pandas as pd
import mysql.connector
from sqlalchemy import create_engine
import os
from dotenv import load_dotenv

In [22]:
pd.set_option('display.max_columns', None)

In [2]:
load_dotenv()

True

In [3]:
DB_HOST = os.getenv('DB_HOST')
DB_USER = os.getenv('DB_USER')
DB_PASSWORD = os.getenv('DB_PASSWORD')
DB_NAME = os.getenv('DB_NAME')

In [4]:
DATABASE_URI = f'mysql+mysqlconnector://{DB_USER}:{DB_PASSWORD}@{DB_HOST}/{DB_NAME}'
engine = create_engine(DATABASE_URI)

In [5]:
query = '''
    SELECT 
        t.id AS tour_id,
        t.productType,
        t.name AS tour_name,
        t.brand,
        t.activityLevel,
        t.reviewsRating,
        t.reviewsTotal,
        t.lowestOptionRoomType,
        t.lowestOptionPrice,
        t.lowestOptionFullPrice,
        t.fkSeasonId,

        o.id AS tour_option_id,
        o.tour_id,
        o.sourceTourOptionName,
        o.isPrivateRequest,
        o.maxPax,
        o.name AS tour_option_name,
        o.description,
        o.travelInclusions,
        o.diningInclusions,
        o.startLocationName,
        o.startLocationCountryCode,
        o.startLocationLongitude,
        o.startLocationLatitude,
        o.endLocationName,
        o.endLocationCountryCode,
        o.endLocationLongitude,
        o.endLocationLatitude,
        o.countries_visited,
        o.minChildPriceAge,
        o.maxChildPriceAge

    FROM tours t
    INNER JOIN tour_options o ON t.id = o.tour_id;

'''

In [6]:
data = pd.read_sql(query, engine)

In [39]:
data

,tour_id,productType,tour_name,brand,activityLevel,reviewsRating,reviewsTotal,lowestOptionRoomType,lowestOptionPrice,lowestOptionFullPrice,fkSeasonId,tour_option_id,tour_id,sourceTourOptionName,isPrivateRequest,maxPax,tour_option_name,description,travelInclusions,diningInclusions,startLocationName,startLocationCountryCode,startLocationLongitude,startLocationLatitude,endLocationName,endLocationCountryCode,endLocationLongitude,endLocationLatitude,countries_visited,minChildPriceAge,maxChildPriceAge,day_1_title,day_1_description,day_1_accommodation,day_1_meals,day_2_title,day_2_description,day_2_accommodation,day_2_meals,day_3_title,day_3_description,day_3_accommodation,day_3_meals,day_4_title,day_4_description,day_4_accommodation,day_4_meals,day_5_title,day_5_description,day_5_accommodation,day_5_meals,day_6_title,day_6_description,day_6_accommodation,day_6_meals,day_7_title,day_7_description,day_7_accommodation,day_7_meals,day_8_title,day_8_description,day_8_accommodation,day_8_meals,day_9_title,day_9_description,day_9_accommodation,day_9_meals,day_10_title,day_10_description,day_10_accommodation,day_10_meals,day_11_title,day_11_description,day_11_accommodation,day_11_meals,day_12_title,day_12_description,day_12_accommodation,day_12_meals,day_13_title,day_13_description,day_13_accommodation,day_13_meals,day_14_title,day_14_description,day_14_accommodation,day_14_meals,day_15_title,day_15_description,day_15_accommodation,day_15_meals,day_16_title,day_16_description,day_16_accommodation,day_16_meals,day_17_title,day_17_description,day_17_accommodation,day_17_meals,day_18_title,day_18_description,day_18_accommodation,day_18_meals,day_19_title,day_19_description,day_19_accommodation,day_19_meals,day_20_title,day_20_description,day_20_accommodation,day_20_meals,day_21_title,day_21_description,day_21_accommodation,day_21_meals,day_22_title,day_22_description,day_22_accommodation,day_22_meals,day_23_title,day_23_description,day_23_accommodation,day_23_meals,day_24_title,day_24_description,day_24_accommodation,day_24_meals,day_25_title,day_25_description,day_25_accommodation,day_25_meals,day_26_title,day_26_description,day_26_accommodation,day_26_meals,day_27_title,day_27_description,day_27_accommodation,day_27_meals,day_28_title,day_28_description,day_28_accommodation,day_28_meals
0,tour-50422032-fd11-4b38-9334-09b67d61270c,connection_tour,Majestic Alaska,trafalgar,None,10.0,2,twin,4405.5,4895.0,06cfb7cd-f9f1-4e90-b9c1-28b6476d7a8a,0044857c-35c9-45f4-97b8-75e7eb7e90f6,tour-50422032-fd11-4b38-9334-09b67d61270c,None,0,NaN,Majestic Alaska,A wild escape brings your dream of Alaska to l...,"Must-see Highlights\n- Explore Valdez, Seward,...",Whats Included\n- An expert Travel Director an...,Anchorage,None,NaN,NaN,Anchorage,None,NaN,NaN,United States,5,17,Start Your Adventure In Anchorage,Welcome to the edge of the wilderness – to a c...,Sheraton Anchorage,Welcome Reception,Step Into The Wilderness Of Denali National Park,Kickstart your day with a brief Anchorage City...,Denali Park Village,Breakfast,Uncover The Natural History Of Denali,With the Denali National Park and Preserve jus...,Denali Park Village,Breakfast,Explore Denali And Venture To Fairbanks,Tune into your beautiful surrounds as you’ll h...,River's Edge Resort,"Breakfast, Dinner",Dig Deep Into Alaskan Culture Dive Into Cultur...,"After breakfast, you will enjoy a city tour of...",River's Edge Resort,"Breakfast, Dinner",Take A Scenic Journey To Valdez,We journey south to the port community of Vald...,Best Western Valdez Harbor Inn,Breakfast,Spot The Wildlife Of Valdez,"Delve into the heritage and culture of Valdez,...",Best Western Valdez Harbor Inn,"Breakfast, Lunch",Travel Back To Anchorage Make Travel Matter,We begin our return journey to Anchorage this ...,Sheraton Anchorage,"Breakfast, Dinner",Glance At Glaciers Make Travel Matter,We journey along the Scenic Seward Byway this ...,Sheraton Anchorage,"Breakfast, Lunch",Depart Anchorage,Our adventure through the rugged Alaskan 

In [41]:
itinerary_joins = []
itinerary_selects = []
for day in range(1, 29):  # 1 to 28
    join = f"LEFT JOIN itineraries i{day} ON o.id = i{day}.tour_option_id AND i{day}.startDay = {day}"
    selects = [
        f"i{day}.title AS day_{day}_title",
        f"i{day}.description AS day_{day}_description",
        f"i{day}.accommodation AS day_{day}_accommodation",
        f"i{day}.meals AS day_{day}_meals",
    ]
    itinerary_joins.append(join)
    itinerary_selects.extend(selects)
    
itinerary_joins_sql = "\n ".join(itinerary_joins)
itinerary_selects_sql = ",\n ".join(itinerary_selects)


In [24]:
query_extended = f'''
    SELECT 
        t.id AS tour_id,
        t.productType,
        t.name AS tour_name,
        t.brand,
        t.activityLevel,
        t.reviewsRating,
        t.reviewsTotal,
        t.lowestOptionRoomType,
        t.lowestOptionPrice,
        t.lowestOptionFullPrice,
        t.fkSeasonId,

        o.id AS tour_option_id,
        o.tour_id,
        o.sourceTourOptionName,
        o.isPrivateRequest,
        o.maxPax,
        o.name AS tour_option_name,
        o.description,
        o.travelInclusions,
        o.diningInclusions,
        o.startLocationName,
        o.startLocationCountryCode,
        o.startLocationLongitude,
        o.startLocationLatitude,
        o.endLocationName,
        o.endLocationCountryCode,
        o.endLocationLongitude,
        o.endLocationLatitude,
        o.countries_visited,
        o.minChildPriceAge,
        o.maxChildPriceAge,

        {itinerary_selects_sql}
        
        FROM tours t
        INNER JOIN tour_options o ON t.id = o.tour_id
    
        {itinerary_joins_sql}

;'''

In [25]:
data = pd.read_sql(query_extended, engine)
data

,tour_id,productType,tour_name,brand,activityLevel,reviewsRating,reviewsTotal,lowestOptionRoomType,lowestOptionPrice,lowestOptionFullPrice,fkSeasonId,tour_option_id,tour_id,sourceTourOptionName,isPrivateRequest,maxPax,tour_option_name,description,travelInclusions,diningInclusions,startLocationName,startLocationCountryCode,startLocationLongitude,startLocationLatitude,endLocationName,endLocationCountryCode,endLocationLongitude,endLocationLatitude,countries_visited,minChildPriceAge,maxChildPriceAge,day_1_title,day_1_description,day_1_accommodation,day_1_meals,day_2_title,day_2_description,day_2_accommodation,day_2_meals,day_3_title,day_3_description,day_3_accommodation,day_3_meals,day_4_title,day_4_description,day_4_accommodation,day_4_meals,day_5_title,day_5_description,day_5_accommodation,day_5_meals,day_6_title,day_6_description,day_6_accommodation,day_6_meals,day_7_title,day_7_description,day_7_accommodation,day_7_meals,day_8_title,day_8_description,day_8_accommodation,day_8_meals,day_9_title,day_9_description,day_9_accommodation,day_9_meals,day_10_title,day_10_description,day_10_accommodation,day_10_meals,day_11_title,day_11_description,day_11_accommodation,day_11_meals,day_12_title,day_12_description,day_12_accommodation,day_12_meals,day_13_title,day_13_description,day_13_accommodation,day_13_meals,day_14_title,day_14_description,day_14_accommodation,day_14_meals,day_15_title,day_15_description,day_15_accommodation,day_15_meals,day_16_title,day_16_description,day_16_accommodation,day_16_meals,day_17_title,day_17_description,day_17_accommodation,day_17_meals,day_18_title,day_18_description,day_18_accommodation,day_18_meals,day_19_title,day_19_description,day_19_accommodation,day_19_meals,day_20_title,day_20_description,day_20_accommodation,day_20_meals,day_21_title,day_21_description,day_21_accommodation,day_21_meals,day_22_title,day_22_description,day_22_accommodation,day_22_meals,day_23_title,day_23_description,day_23_accommodation,day_23_meals,day_24_title,day_24_description,day_24_accommodation,day_24_meals,day_25_title,day_25_description,day_25_accommodation,day_25_meals,day_26_title,day_26_description,day_26_accommodation,day_26_meals,day_27_title,day_27_description,day_27_accommodation,day_27_meals,day_28_title,day_28_description,day_28_accommodation,day_28_meals
0,tour-50422032-fd11-4b38-9334-09b67d61270c,connection_tour,Majestic Alaska,trafalgar,None,10.0,2,twin,4405.5,4895.0,06cfb7cd-f9f1-4e90-b9c1-28b6476d7a8a,0044857c-35c9-45f4-97b8-75e7eb7e90f6,tour-50422032-fd11-4b38-9334-09b67d61270c,None,0,NaN,Majestic Alaska,A wild escape brings your dream of Alaska to l...,"Must-see Highlights\n- Explore Valdez, Seward,...",Whats Included\n- An expert Travel Director an...,Anchorage,None,NaN,NaN,Anchorage,None,NaN,NaN,United States,5,17,Start Your Adventure In Anchorage,Welcome to the edge of the wilderness – to a c...,Sheraton Anchorage,Welcome Reception,Step Into The Wilderness Of Denali National Park,Kickstart your day with a brief Anchorage City...,Denali Park Village,Breakfast,Uncover The Natural History Of Denali,With the Denali National Park and Preserve jus...,Denali Park Village,Breakfast,Explore Denali And Venture To Fairbanks,Tune into your beautiful surrounds as you’ll h...,River's Edge Resort,"Breakfast, Dinner",Dig Deep Into Alaskan Culture Dive Into Cultur...,"After breakfast, you will enjoy a city tour of...",River's Edge Resort,"Breakfast, Dinner",Take A Scenic Journey To Valdez,We journey south to the port community of Vald...,Best Western Valdez Harbor Inn,Breakfast,Spot The Wildlife Of Valdez,"Delve into the heritage and culture of Valdez,...",Best Western Valdez Harbor Inn,"Breakfast, Lunch",Travel Back To Anchorage Make Travel Matter,We begin our return journey to Anchorage this ...,Sheraton Anchorage,"Breakfast, Dinner",Glance At Glaciers Make Travel Matter,We journey along the Scenic Seward Byway this ...,Sheraton Anchorage,"Breakfast, Lunch",Depart Anchorage,Our adventure through the rugged Alaskan 

In [28]:
itineraries_query = "SELECT id, tour_option_id FROM itineraries"
itineraries_df = pd.read_sql(itineraries_query, engine)
itineraries_df

,id,tour_option_id
0,4857,0044857c-35c9-45f4-97b8-75e7eb7e90f6
1,4858,0044857c-35c9-45f4-97b8-75e7eb7e90f6
2,4859,0044857c-35c9-45f4-97b8-75e7eb7e90f6
3,4860,0044857c-35c9-45f4-97b8-75e7eb7e90f6
4,4861,0044857c-35c9-45f4-97b8-75e7eb7e90f6
...,...,...
8945,3283,fffbe628-ba84-4d9a-99c6-d2c44edf3a52
8946,3284,fffbe628-ba84-4d9a-99c6-d2c44edf3a52
8947,3285,fffbe628-ba84-4d9a-99c6-d2c44edf3a52
8948,3286,fffbe628-ba84-4d9a-99c6-d2c44edf3a52


In [34]:
# Fetching locations
locations_query = "SELECT itinerary_id, name, countryCode, longitude, latitude FROM locations_visited"
locations_df = pd.read_sql(locations_query, engine)
display(locations_df)
locations_dict = locations_df.groupby('itinerary_id').apply(lambda x: x[['name', 'countryCode', 'longitude', 'latitude']].to_dict(orient='records')).to_dict()
# locations_dict

,itinerary_id,name,countryCode,longitude,latitude
0,1,Hanoi,VN,105.699772,20.998488
1,2,Hanoi,VN,105.699772,20.998488
2,2,Ha Long Bay,VN,107.051479,20.824062
3,3,Hanoi,VN,105.699772,20.998488
4,4,Ho Chi Minh City,VN,106.713355,10.743635
...,...,...,...,...,...
16560,8934,Salzburg,AT,13.046118,47.797631
16561,8935,Berlin,DE,13.236281,52.436959
16562,8936,Innsbruck,AT,11.380083,47.289626
16563,8936,Salzburg,AT,13.046118,47.797631


In [35]:
merged_locations_df = pd.merge(locations_df, itineraries_df, left_on='itinerary_id', right_on='id', suffixes=('_location', '_itinerary'))


In [36]:
locations_dict = merged_locations_df.groupby('tour_option_id').apply(lambda x: x[['name', 'countryCode', 'longitude', 'latitude']].to_dict(orient='records')).to_dict()


In [40]:
data['locations'] = data['tour_option_id'].map(locations_dict)
data

,tour_id,productType,tour_name,brand,activityLevel,reviewsRating,reviewsTotal,lowestOptionRoomType,lowestOptionPrice,lowestOptionFullPrice,fkSeasonId,tour_option_id,tour_id,sourceTourOptionName,isPrivateRequest,maxPax,tour_option_name,description,travelInclusions,diningInclusions,startLocationName,startLocationCountryCode,startLocationLongitude,startLocationLatitude,endLocationName,endLocationCountryCode,endLocationLongitude,endLocationLatitude,countries_visited,minChildPriceAge,maxChildPriceAge,day_1_title,day_1_description,day_1_accommodation,day_1_meals,day_2_title,day_2_description,day_2_accommodation,day_2_meals,day_3_title,day_3_description,day_3_accommodation,day_3_meals,day_4_title,day_4_description,day_4_accommodation,day_4_meals,day_5_title,day_5_description,day_5_accommodation,day_5_meals,day_6_title,day_6_description,day_6_accommodation,day_6_meals,day_7_title,day_7_description,day_7_accommodation,day_7_meals,day_8_title,day_8_description,day_8_accommodation,day_8_meals,day_9_title,day_9_description,day_9_accommodation,day_9_meals,day_10_title,day_10_description,day_10_accommodation,day_10_meals,day_11_title,day_11_description,day_11_accommodation,day_11_meals,day_12_title,day_12_description,day_12_accommodation,day_12_meals,day_13_title,day_13_description,day_13_accommodation,day_13_meals,day_14_title,day_14_description,day_14_accommodation,day_14_meals,day_15_title,day_15_description,day_15_accommodation,day_15_meals,day_16_title,day_16_description,day_16_accommodation,day_16_meals,day_17_title,day_17_description,day_17_accommodation,day_17_meals,day_18_title,day_18_description,day_18_accommodation,day_18_meals,day_19_title,day_19_description,day_19_accommodation,day_19_meals,day_20_title,day_20_description,day_20_accommodation,day_20_meals,day_21_title,day_21_description,day_21_accommodation,day_21_meals,day_22_title,day_22_description,day_22_accommodation,day_22_meals,day_23_title,day_23_description,day_23_accommodation,day_23_meals,day_24_title,day_24_description,day_24_accommodation,day_24_meals,day_25_title,day_25_description,day_25_accommodation,day_25_meals,day_26_title,day_26_description,day_26_accommodation,day_26_meals,day_27_title,day_27_description,day_27_accommodation,day_27_meals,day_28_title,day_28_description,day_28_accommodation,day_28_meals,locations
0,tour-50422032-fd11-4b38-9334-09b67d61270c,connection_tour,Majestic Alaska,trafalgar,None,10.0,2,twin,4405.5,4895.0,06cfb7cd-f9f1-4e90-b9c1-28b6476d7a8a,0044857c-35c9-45f4-97b8-75e7eb7e90f6,tour-50422032-fd11-4b38-9334-09b67d61270c,None,0,NaN,Majestic Alaska,A wild escape brings your dream of Alaska to l...,"Must-see Highlights\n- Explore Valdez, Seward,...",Whats Included\n- An expert Travel Director an...,Anchorage,None,NaN,NaN,Anchorage,None,NaN,NaN,United States,5,17,Start Your Adventure In Anchorage,Welcome to the edge of the wilderness – to a c...,Sheraton Anchorage,Welcome Reception,Step Into The Wilderness Of Denali National Park,Kickstart your day with a brief Anchorage City...,Denali Park Village,Breakfast,Uncover The Natural History Of Denali,With the Denali National Park and Preserve jus...,Denali Park Village,Breakfast,Explore Denali And Venture To Fairbanks,Tune into your beautiful surrounds as you’ll h...,River's Edge Resort,"Breakfast, Dinner",Dig Deep Into Alaskan Culture Dive Into Cultur...,"After breakfast, you will enjoy a city tour of...",River's Edge Resort,"Breakfast, Dinner",Take A Scenic Journey To Valdez,We journey south to the port community of Vald...,Best Western Valdez Harbor Inn,Breakfast,Spot The Wildlife Of Valdez,"Delve into the heritage and culture of Valdez,...",Best Western Valdez Harbor Inn,"Breakfast, Lunch",Travel Back To Anchorage Make Travel Matter,We begin our return journey to Anchorage this ...,Sheraton Anchorage,"Breakfast, Dinner",Glance At Glaciers Make Travel Matter,We journey along the Scenic Seward Byway this ...,Sheraton Anchorage,"Breakfast, Lunch",Depart Anchorage,Our adventure through the rugge

In [44]:
directory = "../../data/processed"
file_path = os.path.join(directory, f"merged.csv")
data.to_csv(file_path, index=False)